![Podmanlogo](Pictures/podman-logo.png)

# Managing pods

All this time we've been working with simple containers, but Podman received its name because it's also a pod manager. But, what are a pods? Pods are a group of one or more containers sharing the same network, pid and ipc namespaces. If two containers are in the same pod they can reach each other by using localhost or 127.0.0.1 address.

Moreover, pods are the smallest atomic unit that kubernetes clusters can manage. Running pods is super useful to run local tests without the need of deploying them on a kubernetes cluster.

Let's see how to manage pods with Podman.

Start by login in:

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Podman201'] }}

First we are going to create a pod, for the moment it'll be empty.

In [ ]:
podman pod create --name my-pod

We can now see the list of our pods.

In [ ]:
podman pod ls

We can see the list of containers and what pod they are in by running the following command:

In [ ]:
podman ps -a --pod

As you can see there is already a container within our pod. This is the infra container, an infra container is a lightweight container used to coordinate the shared kernel namespace of a pod. This container is created by default, you can overwrite this behaviour but it's not recommended.

We will run a wordpress workload within our pod, we need to run a mariadb container as backend and a wordpress container as frontend. Let's first deploy the database:

In [ ]:
podman run -d --pod my-pod \
  -e MYSQL_ROOT_PASSWORD="hpedev" \
  -e MYSQL_DATABASE="wp" \
  -e MYSQL_USER="wordpress" \
  -e MYSQL_PASSWORD="wordpress" \
  --name wp-db \
  docker.io/library/mariadb:latest

There are a couple of important points to mention regarding previous command. First, we have specified that our container should be part of our pod with the option "--pod". Second we are passing environment variables to the container with the option "-e", by doing this we can do some basic configuration for this instance of the container.

If we list the containers and pods again we should see two containers in the same pod.

In [ ]:
podman ps -a --pod

We can also use the following command:

In [ ]:
podman pod ps

Let's add the second container, the fronend.

In [ ]:
podman run -d  --pod=my-pod \
  -e WORDPRESS_DB_NAME="wp" \
  -e WORDPRESS_DB_USER="wordpress" \
  -e WORDPRESS_DB_PASSWORD="wordpress" \
  -e WORDPRESS_DB_HOST="127.0.0.1" \
  --name wp-web \
  docker.io/library/wordpress:latest

As you can see we've used the address 127.0.0.1 for the environment variable WORDPRESS_DB_HOST, this means we're pointing to localhost for finding the database. This is only possible because both containers (the web and the database) are running in the same pod.

Checking the logs we can see there is no issue with the database, confirming it works properly:

In [ ]:
podman logs wp-web

Before looking at the patient portal application let’s first clean up:

In [ ]:
podman rm --all -f
podman network prune -f
podman volume prune -f
podman pod prune -f
podman image prune -f

# Deploy Patient Portal application using pods

This time we are going to demonstrate how to deploy the patient portal application using pods. For this we will have 1 pod with the frontend and database containers. As both containers will be in the same pod they share the same network, hence we'll need a single Podman network for all of our application.

![PatientPortalApplication](Pictures/patient-portal-application-pod.png)

Start creating the network:

In [ ]:
podman network create patient-portal-net

We can now create our Pod, we'll connect it to our newly created network. What's most important we'll not be exposing ports at the container level, we do it at Pod level:

In [ ]:
podman pod create --name patient-portal-pod --net patient-portal-net -p 8080:8080

As you can see we exposed the port 8080 which is the one used by the frontend container but we did not expose the port 5432 which is the one used by the database. This is because the traffic between frontend and database will happen internally in the Pod.

Create the database container in our newly created Pod:

In [ ]:
podman run -d --rm --name database --pod patient-portal-pod quay.io/skupper/patient-portal-database

Lets create the payment processor container outside the Pod:

In [ ]:
podman run -d --rm --name payment-processor --net patient-portal-net quay.io/skupper/patient-portal-payment-processor

We can review our containers and pods:

In [ ]:
podman ps -a --pod

Everything is ready, it's time to deploy our frontend container inside our Pod. Pay attention to the environment variables, we pass "localhost" as the value for the variable that defines the database hostname. As the frontend will reach the database inside the pod it's considered local traffic (localhost or 127.0.0.1).

In [ ]:
podman run -d --rm --name frontend --pod patient-portal-pod \
-e DATABASE_SERVICE_HOST="localhost" \
-e DATABASE_SERVICE_PORT="5432" \
-e PAYMENT_PROCESSOR_SERVICE_HOST="payment-processor" \
-e PAYMENT_PROCESSOR_SERVICE_PORT="8080" \
quay.io/skupper/patient-portal-frontend

Our application is now deployed, check everything is fine by looking at frontend logs:

In [ ]:
podman logs frontend

Check the application is reachable through the open port in the pod:

In [ ]:
curl -s localhost:8080

Last, take a look at the output of this command:

In [ ]:
podman ps --filter network=patient-portal-net

As you can see it shows that only the infra container in the pod is connected to the network. This is because this container is the one managing the internal network of the pod. Hence if you want to expose anything into any network you need to do it through this container and not through your application microservices container. This is exactly what you see: Our "database" and "frontend" don't have any port exposed and neither they are connected to any network directly, everything is managed through the pod.

# Cleanup

In [ ]:
podman rm --all -f
podman network prune -f
podman volume prune -f
podman pod prune -f
podman image prune -f

In [ ]:
%logout

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#631f61;"></i>&nbsp;&nbsp;Next Steps

# Lab 4 : Running Containers

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="4-WKSHP-Running-containers-at-system-start.ipynb" target="New" title="Next LAB: Running containers at system start"><i class="fas fa-chevron-circle-right" style="color:#631f61;"></i></a></h2>

</br>
 <a href="3-WKSHP-Managing-pods.ipynb" target="New" title="Back: Managing Pods"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="4-WKSHP-Running-containers-at-system-start.ipynb" target="New" title="Next:Running containers at system start"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
